In [1]:
import pandas as pd
import numpy as np

# Lecture du dataset horodateur
dfh = pd.read_csv('horodateurs-mobiliers.csv', sep = ';')


In [2]:
monRepertoire= 'D:/BUREAU/corsESGI/Projet_annuel/transactions_horodateur/partition_horodateur/'
destination_rep="D:\\BUREAU\\corsESGI\\Projet_annuel\\Last_datasets\\"
path = monRepertoire

In [3]:
# Liste des nom des fichiers horodateur
from os import listdir
from os.path import isfile, join

fichiers = [f for f in listdir(monRepertoire) if isfile(join(monRepertoire, f))]
fichiers.sort(key=len)



In [4]:
tranche = ['00:00:00-00:30:00','00:30:00-01:00:00','01:00:00-01:30:00','01:30:00-02:00:00','02:00:00-02:30:00','02:30:00-03:00:00','03:00:00-03:30:00','03:30:00-04:00:00','04:00:00-04:30:00','04:30:00-05:00:00','05:00:00-05:30:00','05:30:00-06:00:00','06:00:00-06:30:00','06:30:00-07:00:00','07:00:00-07:30:00','07:30:00-08:00:00','08:00:00-08:30:00','08:30:00-09:00:00','09:00:00-09:30:00','09:30:00-10:00:00','10:00:00-10:30:00','10:30:00-11:00:00','11:00:00-11:30:00','11:30:00-12:00:00','12:00:00-12:30:00','12:30:00-13:00:00','13:00:00-13:30:00','13:30:00-14:00:00','14:00:00-14:30:00','14:30:00-15:00:00','15:00:00-15:30:00','15:30:00-16:00:00','16:00:00-16:30:00','16:30:00-17:00:00','17:00:00-17:30:00','17:30:00-18:00:00','18:00:00-18:30:00','18:30:00-19:00:00','19:00:00-19:30:00','19:30:00-20:00:00','20:00:00-20:30:00','20:30:00-21:00:00','21:00:00-21:30:00','21:30:00-22:00:00','22:00:00-22:30:00','22:30:00-23:00:00','23:00:00-23:30:00','23:30:00-00:00:00']

months= ['2014-02','2014-03','2014-04','2014-05','2014-06','2014-07','2014-08','2014-09','2014-10','2014-11','2014-12','2015-02','2015-03','2015-04','2015-05','2015-06','2015-07','2015-08','2015-09','2015-10','2015-11','2015-12']
  
month = '2014-01'
period = pd.Period(month, freq='M')
tmp=pd.Series(pd.date_range(start=period.start_time, end=period.end_time, freq='D')).to_frame('date')

for e in months:
    
    month = e
    period = pd.Period(month, freq='M')
    tmp1=pd.Series(pd.date_range(start=period.start_time, end=period.end_time, freq='D')).to_frame('date')
    tmp=pd.concat([tmp, tmp1], ignore_index=True)
year_df=tmp
year_df=pd.concat([pd.DataFrame(columns=tranche),year_df])
year_dh=year_df.fillna(0)
year_dh['date']=pd.to_datetime(year_df["date"]) 

days_of_week = ['Mondy','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']


C:\Users\merzo\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [5]:
def strr(x):
    return x.strftime("%Y-%m-%d")
def strSpace(str):
    return str.replace(' ','')

In [6]:
def get_lines_different_dates(pp):
    for index,row in pp.iterrows():
        
        heure_fin=row['heure_fin_stationnement']
        heure_debut=row['heure_debut_stationnement']
        
#         ici j'insere une ligne en mettant la date de fin = date debut et l'heure de fin à 23:59:59
#         première ligne 
        deb= datetime.datetime.strptime(row['date_heure_debut_stationnement'], '%Y-%m-%d %H:%M:%S')
        fin=datetime.datetime.strptime(row['date_heure_fin_stationnement'], '%Y-%m-%d %H:%M:%S')
#       on genère une liste de date comprises entre date debut et date fin
        my_l=pd.date_range(start=deb.date(), end=fin.date()).tolist()
    
        
        pli=pd.DataFrame(my_l,columns=['date_debut_stationnement']) 
        pli['date_fin_stationnement']=my_l
        pli['date_debut_stationnement']=pli['date_debut_stationnement'].apply(strr)
        pli['date_fin_stationnement']=pli['date_fin_stationnement'].apply(strr)
       
        pli['heure_debut_stationnement'] ='00:00:00'
        pli['heure_debut_stationnement'][0]=heure_debut
        pli['heure_fin_stationnement'] = '23:59:59'
        pli['heure_fin_stationnement'][len(pli['heure_fin_stationnement'].tolist())-1]=heure_fin

        pp=pd.concat([pp,pli],ignore_index=True)
        
        pp= pp[pp['date_debut_stationnement'] ==pp['date_fin_stationnement'] ]   
        
    return pp


In [7]:
# Ajout des colonnes N° d'horodateur","Zone résidentielle","Adresse","geo_shape","geo_point_2d","date",
# "heure" et "date_heure" ( debut et fin de stationnement) aux fichiers horodateurs  


import datetime

#  Arrêté sur le 338/ 611 

# on parcour la liste des fichiers horodateur
i=1

# merzouk fichiers[:2603], aghylas fichiers[2603:5206] et juba fichiers [5206;7809]
fichiers_batch=fichiers[:2603]
for e in fichiers_batch:
    print(i,'/',len(fichiers_batch))

#     on charge les données, création des colonnes date, heure ..
    tmp = pd.read_csv(path+e, sep =';')
    tmp2= pd.merge(tmp,dfh.filter(["N° d'horodateur","Zone résidentielle","Adresse","geo_shape","geo_point_2d"]) , left_on='horodateur', right_on="N° d'horodateur")
    del tmp2["N° d'horodateur"]
    
    # creation des colonnes vides
    tmp2["date_heure_debut_stationnement"] = tmp2.apply(lambda _: '', axis=1)
    tmp2["date_debut_stationnement"] = tmp2.apply(lambda _: '', axis=1)
    tmp2["heure_debut_stationnement"] = tmp2.apply(lambda _: '', axis=1)
    
    tmp2["date_heure_fin_stationnement"] = tmp2.apply(lambda _: '', axis=1)
    tmp2["date_fin_stationnement"] = tmp2.apply(lambda _: '', axis=1)
    tmp2["heure_fin_stationnement"] = tmp2.apply(lambda _: '', axis=1)
    h=tmp2['horodateur'][0]
    geo_point=tmp2['geo_point_2d'][0]
    
    # Remplissage des colonnes
#     réccuperation des dates et heure au format yyyy-mm-dd     et hh:mm:ss
    tmp2["date_heure_debut_stationnement"] = tmp2["debut_stationnement"].apply(lambda x: x.replace('T',' ').split('+')[0])
    tmp2["date_debut_stationnement"]  = tmp2["debut_stationnement"].apply(lambda x: x.replace('T',' ').split('+')[0].split(' ')[0] ) 
    tmp2["heure_debut_stationnement"] = tmp2["debut_stationnement"].apply(lambda x: x.replace('T',' ').split('+')[0].split(' ')[1] ) 
   
    tmp2["date_heure_fin_stationnement"] =tmp2["fin_stationnement"].apply(lambda x: x.replace('T',' ').split('+')[0])
    tmp2["date_fin_stationnement"]  = tmp2["fin_stationnement"].apply(lambda x: x.replace('T',' ').split('+')[0].split(' ')[0] ) 
    tmp2["heure_fin_stationnement"] = tmp2["fin_stationnement"].apply(lambda x: x.replace('T',' ').split('+')[0].split(' ')[1] ) 
    tmp2['date_debut_stationnement']=tmp2['date_debut_stationnement'].apply(strSpace)
    tmp2['date_fin_stationnement']=tmp2['date_fin_stationnement'].apply(strSpace)
    pp=tmp2

#     on prend les lignes date debu != date de fin pour les traiter
    pp=pp[pp['date_debut_stationnement'] <pp['date_fin_stationnement'] ]

    
#      appel de la fonction pour le traitement des date debut != date fin
    pp=get_lines_different_dates(pp)
    

#   on garde les date deb = date fin dans la dataframe tmp2 
    tmp2=tmp2[tmp2['date_debut_stationnement'] ==tmp2['date_fin_stationnement'] ]

#   on concat les data frames, dans pp les nouvelles lignes inserrées
    tmp2 = pd.concat([pp,tmp2], ignore_index=True)
    
#  insertion des colonnes tranches horraire
    for t in tranche:
        tmp2[t] = tmp2.apply(lambda _: 0, axis=1)
        tmp2.loc[((tmp2['heure_debut_stationnement'] <=t.split('-')[0])  &  (tmp2['heure_fin_stationnement'] >=t.split('-')[0] ) | (tmp2['heure_debut_stationnement'] <=t.split('-')[1])  &  (tmp2['heure_fin_stationnement'] >=t.split('-')[1] ) | (tmp2['heure_debut_stationnement'] >=t.split('-')[0])  &  (tmp2['heure_fin_stationnement'] <=t.split('-')[1] )    )& (tmp2["date_debut_stationnement"] == tmp2["date_fin_stationnement"] ),t] =1

    
#     to string sur les colonnes dates
    tmp2['date_debut_stationnement']=tmp2['date_debut_stationnement'].apply(strSpace)
    tmp2['date_fin_stationnement']=tmp2['date_fin_stationnement'].apply(strSpace)
    tmp2=tmp2.reset_index()
    del tmp2['index']
 
    kk=tmp2.groupby(['date_debut_stationnement']).sum()[sorted(tranche, reverse=False)]
    kk=kk.reset_index()
    
    
#     ce bloc c'est pour créer des lignes en plus pour les date n ayant pas de transaction 
#     dans r une liste de tout le jour du 2014-01-01 au 2015-12-12
    r = pd.date_range(start=year_df.date.min(), end=year_df.date.max())
    r=r.strftime("%Y-%m-%d")
    kk=kk.set_index('date_debut_stationnement').reindex(r).fillna(0).rename_axis('date').reset_index()

#     kk['annee']=kk['date'].apply(lambda x : x[:4])
   
    kk['day_of_week']=kk['date'].apply(lambda x :pd.to_datetime(x).weekday() )
#     del kk['date']
    kk['Latitude']=geo_point.split(',')[0]
    kk['Longitude']=geo_point.split(',')[1]
    kk['horodateur']=h
    kk[tranche]=kk[tranche].astype(int)
#     pour ordonner les colonnes
    cols = kk.columns.tolist()
    cols= [cols[-1]]+[cols[-2]]+[cols[-3]]+[cols[0]]+[cols[-4]]+cols[1:-4]
    kk=kk[cols]

    
#     sauvgarde des fichiers pour chaque horodateur
    kk.to_csv(destination_rep+e, index=False, sep = ';')
    i=i+1
    

    

1 / 274


C:\Users\merzo\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\merzo\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



2 / 274


C:\Users\merzo\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\merzo\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



3 / 274
4 / 274
5 / 274
6 / 274
7 / 274
8 / 274
9 / 274
10 / 274
11 / 274
12 / 274
13 / 274
14 / 274
15 / 274
16 / 274
17 / 274
18 / 274
19 / 274
20 / 274
21 / 274
22 / 274
23 / 274
24 / 274
25 / 274
26 / 274
27 / 274
28 / 274
29 / 274
30 / 274
31 / 274
32 / 274
33 / 274
34 / 274
35 / 274
36 / 274
37 / 274
38 / 274
39 / 274
40 / 274
41 / 274
42 / 274
43 / 274
44 / 274
45 / 274
46 / 274
47 / 274
48 / 274
49 / 274
50 / 274
51 / 274
52 / 274
53 / 274
54 / 274
55 / 274
56 / 274
57 / 274
58 / 274
59 / 274
60 / 274
61 / 274
62 / 274
63 / 274
64 / 274
65 / 274
66 / 274
67 / 274
68 / 274
69 / 274
70 / 274
71 / 274
72 / 274
73 / 274
74 / 274
75 / 274
76 / 274
77 / 274
78 / 274
79 / 274
80 / 274
81 / 274
82 / 274
83 / 274
84 / 274
85 / 274
86 / 274
87 / 274
88 / 274
89 / 274
90 / 274
91 / 274
92 / 274
93 / 274
94 / 274
95 / 274
96 / 274
97 / 274
98 / 274
99 / 274
100 / 274
101 / 274
102 / 274
103 / 274
104 / 274
105 / 274
106 / 274
107 / 274
108 / 274
109 / 274
110 / 274
111 / 274
112 / 274
113 